# 最尤推定によるバネの特性計測

今回は，最尤推定の例題としてバネの特性を計測する問題を取り上げ，どのように特性の推定が行われるかについて確認してみます．


# 計測データの作成
まず，バネの計測データを作成します．実際に計測を行うと大変なため，コンピュータ上で仮想的に計測データを作成します．

今回は重さが10gから50gまでの分銅が無数にあるとして，それぞれの重さの分銅を吊るした場合のバネの長さを計測してみます．ただし，今回は長さを正確に計測するための定規を準備することができなかったため，長さの計測は目測で行うものとします．また，このような目測による誤差は，正規分布に従うものとします．

まず，重さ$x$のおもりをバネに吊るした場合のバネの長さは以下の式で表されます．

$y = k x + c$

$k$はバネ定数，$c$は0gのおもりを吊るした場合のバネの長さです．今回はこのバネ定数$k$と長さ$c$の両方がわからないものとして，この２つの推定を行います．

最初に書いたとおり，バネの計測結果は計測誤差を含みますので，計測により得られるバネの長さは以下のように表現できます．

$y_i = k x_i + c_i + \epsilon_i,\ \  \epsilon_i \sim {\cal N}(0,\sigma^2)$

$\sigma$は誤差の標準偏差です．
まずは，この計測モデルに従って$N$回分の計測データを作成します．

In [49]:
#@title 初期設定（左のボタンで実行してください）
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D

k_ans = 3
c_ans = 2
sigma = 0.1



In [ ]:
N = 1000 #　計測回数

weight = np.array([]) # おもりの重さを保存しておく配列の初期化，これに順番に追加していく
length = np.array([]) # 計測された長さを保存しておく配列，計測結果をこれに追加していく

for i in range(N): # N回の繰り返し処理を行うための記述，iが 0からN-1まで順番に変化し，その間以下の命令が実行される
  omori = random.random() * 0.04 + 0.01 # 0.01 kg から 0.05 kg までの数字をランダムに発生させる（rand.random()が0 から 1 までの数字を発生させるので，それを0.4 倍して 0.1 足す
  nagasa = omori * k_ans + c_ans + random.normalvariate(0,sigma) # ここに，上のモデルにしたがって，計測結果を記入する（ここを空欄）

  weight = np.append(weight, omori) # weight に今回のおもりの重さを追加
  length = np.append(length, nagasa) # length に今回の計測結果を追加

# 計測結果をグラフとして表示
plt.scatter(weight, length)

# $k$, $c$に対する尤度の計算
次に，与えられた計測結果から，あるバネ定数$k$と長さ$c$に関する尤度を計算してみます．この計算は何度も記述する必要がありますので，関数 L という形で定義しておきます．

In [60]:
def L(k, c, weight, length, data_num):
  l = 1
  for i in range(0,data_num,1): # 繰り返し処理によりΠ計算を実現
    l *= np.exp(-(length[i]-(weight[i]*k+c))**2) # 一つ一つのデータに対する確率密度を順次掛け算（ここを記述）
  return l

# $k$, $c$を変化させた場合の尤度の計算

先ほどの$L$の定義により，$k$, $c$が与えられれば，その確からしさ（尤度）を計算することができるようになりました．正しい，$k$, $c$を計算するためには，この尤度を最大とする$\hat{k}$, $\hat{c}$を求める必要があります．この推定方法には様々な方法がありますが，今回は$k$, $c$の値を少しずつ変化させて，その数値の中から尤度が最大となるものを探索してみます（このような方法をグリッドサーチと呼びます）．今回は$0<k<5$, $0<c<5$であると仮定し，この範囲で数値を0.5ずつ変化させながら，それぞれの値における尤度を計算してみます．

※この方法は，あまり賢いやり方とはいえません．

In [65]:
L_set = np.array([]) # 尤度を保存しておくための配列
L_max = 0 # 尤度の最大値を保存しておく変数
k_max = 0 # 尤度が最大の場合のkを保存しておく変数
c_max = 0 # 尤度が最大の場合のcを保存しておく変数

for k in np.arange(0,10,0.5): # 0からスタートして，0.5ずつ
  for c in np.arange(0,10,0.5):
    yudo = L(k, c, weight, length, N)
    L_set = np.append(L_set, yudo)
    if L_max < yudo:
      L_max = yudo
      k_max = k
      c_max = c
print("k = %f, c = %f, L = %e" % (k_max, c_max, L_max))

k = 3.000000, c = 2.000000, L = 2.050033e-05


# グラフの表示

ひとまず，尤度を最大とする$k$, $c$を求めることができました．では，次に先ほど求めた尤度が$k$, $c$の値によってどのように変化したか，グラフとして表示してみましょう．

In [ ]:
#@title 尤度のグラフを表示（詳細はちょっと複雑なので説明は割愛，左のボタンを押してください） { display-mode: "form" }
% matplotlib inline
L_set2 = L_set.reshape([20,20])
x_set = np.array([[ i for i in np.arange(0,10,0.5)] for j in np.arange(0,10,0.5)])
y_set = np.array([[ j for i in np.arange(0,10,0.5)] for j in np.arange(0,10,0.5)])

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel("C")
ax.set_ylabel("K")
ax.set_zlabel("L")
ax.plot_surface(np.array(x_set),np.array(y_set),np.array(L_set2))



このグラフからも，求められた$k$, $c$の値をピークとして，尤度の値が減少していることがわかります．この結果から，0.5 刻みでそれぞれの値を変化させた場合に尤度を最大とする$k$, $c$を求めることができました．

しかし，当然$k$, $c$の値は連続的に変化しているため，この値が本当に尤度を最大化しているとは限りません．そこで，最小２乗法を使って$k$, $c$を求めてみます．

# 最小２乗解の推定

バネのモデル式は，以下のように書き換えることができます．

$y = k x + c = 
\left [ \begin{array}{cc} x & 1 \end{array} \right ]
\left [ \begin{array}{c} k \\ c \end{array} \right ]
$

ここで，$N$回の計測が行われているとすると，以下の式を得ることができます．

$
\left [ 
  \begin{array}{c} y_1  \\  \vdots \\ y_N \end{array} 
\right ] = 
\left [ 
  \begin{array}{c} x_1 \ \  1 \\  \vdots \\ x_N \ \  1 \end{array} 
\right ]
\left [ \begin{array}{c} k \\ c \end{array} \right ]
$

これを，以下のように書き換えます．

${\bf Y} = {\bf X} \beta$


指導書にあるとおり，今回の問題の最小2乗解は以下の式で計算されます．

$ {\bf \beta} = ({\bf X}^T{\bf X})^{-1}{\bf X}^T {\bf y}$

それでは，実際に計算してみましょう．


In [67]:
X = np.append([weight], [np.ones(N)],axis=0).transpose() # [x_1, 1], [x_2,1] ,,,, ] の行列を作成
y = length.transpose(); # Y の行列を作成

beta = np.linalg.inv(X.transpose() @ X) @ X.transpose() @ y # ここを記述させる
print("k = %f, c = %f" % (beta[0], beta[1])) # 表示させる．

k = 2.819189, c = 2.003289


# 確認
求められた値の尤度が最大になっているか確認してみましょう

また，少しだけずらした値（0.01足した値）と比較してみて，本当に最大となっているかみてみましょう．

In [68]:
print("L(%f, %f) = %f"%(beta[0], beta[1],L(beta[0], beta[1], weight, length, N)))
print("L(%f, %f) = %f"%(beta[0]+0.01, beta[1]+0.01,L(beta[0]+0.01, beta[1]+0.01, weight, length, N)))


L(2.819189, 2.003289) = 0.000021
L(2.829189, 2.013289) = 0.000019


$k$, $c$の値を少し変化させただけで，尤度が小さくなっていることが確認できました．このことから，最小２乗法によって求められた$k$, $c$が極大値となっていることを確認できました．

今回は$N = 100$ の場合で試してみましたが，$N$の値を大きくすると，尤度はどのように変化するでしょうか？
実際に試してみましょう（やり方はビデオを確認）
どうように，誤差の標準偏差$\sigma$を変化させた場合はどうなるでしょうか．

以上を踏まえて，以下の内容を写真にとって提出してください．

1.   最小2乗法により求められた数字を撮影した写真
2.   N = 500 の場合の尤度のグラフ
3.   N = 1000 の場合の尤度のグラフ
4.  $\sigma=0.01$の場合の$N=500, N=1000$の場合の尤度のグラフ
5. 上記のように尤度が変化した理由に関する考察

今回の演習は以上で終了です．
